# Here multitasking was done by training the same structure with arousal,valence & liking simultaneously 
# Loading the train & test datas

In [ ]:
import os
import fnmatch
import numpy as np

from sys     import argv
#from sklearn import svm

from load_features     import load_all
#from calc_scores       import calc_scores
#from write_predictions import write_predictions

# Set folders here
path_test_predictions = "D:/AVEC_17_Emotion_Sub-Challenge/result/"
b_test_available      = False  # If the test labels are not available, the predictions on test are written into the folder 'path_test_predictions'

# Folders wi o_features = "E:/AVEC_17_Emotion_Sub-Challenge/audio_features_xbow_6s/"
path_audio_features = "D:/AVEC_17_Emotion_Sub-Challenge/audio_features_xbow_6s/"
path_audio_6s = "D:/AVEC_17_Emotion_Sub-Challenge/audio_features_functionals_6s/"
path_video_features = "D:/AVEC_17_Emotion_Sub-Challenge/video_features_xbow_6s/"
path_text_features  = "D:/AVEC_17_Emotion_Sub-Challenge/text_features_xbow_6s/"
path_labels         = "D:/AVEC_17_Emotion_Sub-Challenge/labels/"

sr_labels = 0.1

delay = 2.2
b_audio = True
b_video = True
b_text  = True
rise = 0.20




path_features = []
    
     
    #path_features.append( path_audio_6s )
if b_audio:
    path_features.append( path_audio_features )
        #path_features.append( path_audio_6s )
if b_video:
    path_features.append( path_video_features )
if b_text:
    path_features.append( path_text_features )
        

if not b_test_available and not os.path.exists(path_test_predictions):
    os.mkdir(path_test_predictions)

# Compensate the delay (quick solution)
shift = int(np.round(delay/sr_labels))
shift = np.ones(len(path_features),dtype=int)*shift



files_train = fnmatch.filter(os.listdir(path_features[0]), "Train*")  # Filenames are the same for audio, video, text & labels
files_devel = fnmatch.filter(os.listdir(path_features[0]), "Devel*")
files_test  = fnmatch.filter(os.listdir(path_features[0]), "Test*")





# Load features and labels
    

In [ ]:
Train   = load_all( files_train, path_features, shift )
Devel   = load_all( files_devel, path_features, shift )
Train_L = load_all( files_train, [ path_labels ] )  # Labels are not shifted
Devel_L = load_all( files_devel, [ path_labels ] )

# windowing the timesteps

def create_data(dataset,look_back=1):
    dataX = []
    for i in range(len(dataset)-look_back+1):
        a = dataset[i:i+look_back,:]
        dataX.append(a)
    return np.array(dataX)

In [ ]:
look_back  = 8

New_TrainX = create_data(Train,look_back)
New_TrainY = Train_L[look_back-1:,]
#TrainY[1:3,]

Devel_X = create_data(Devel,look_back)
Devel_Y = Devel_L[look_back-1:,]
    
    
Y = New_TrainY
X =  New_TrainX
    
    
    
np.random.seed(0)
CCC_max = list()
PCC_max = list()

CCC_max1 = list()
PCC_max1 = list()

CCC_max2 = list()
PCC_max2 = list()    

# Creating the network

In [ ]:
    
    
        
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Bidirectional

model = Sequential()
        
model.add(LSTM(600,input_shape=(look_back,4521),return_sequences = True,go_backwards=True))
model.add(LSTM(200,input_shape=(look_back,4521) ,return_sequences = True,go_backwards=True))
model.add(LSTM(200,input_shape=(look_back,4521), return_sequences = True,go_backwards=True))
model.add(LSTM(200,input_shape=(look_back,4521) ,return_sequences = True,go_backwards=True))
model.add(LSTM(200,input_shape=(look_back,4521), return_sequences = True,go_backwards=True))
model.add(LSTM(200,input_shape=(look_back,4521), return_sequences = True,go_backwards=True))
model.add(LSTM(200,input_shape=(look_back,4521),go_backwards=True))
model.add(Dense(3, activation= 'tanh' ))

model.compile(loss = 'mean_squared_error',optimizer = 'adam',metrics = ['accuracy'])
print(model.summary())
        
        
        

# Evaluation

In [ ]:
model.fit(X,Y,epochs = 1,validation_data=(Devel_X, Devel_Y))

# Calculating CCC,PCC & RMSE

In [ ]:
    
pred = model.predict(Devel_X)
        
pred_A = pred[:,0]
pred_V = pred[:,1]
pred_L = pred[:,2]

# CCC & PCC for arousal

In [ ]:
    
x = pred_A.reshape(len(Devel_Y),)


y = Devel_Y[:,0]

x_mean = np.nanmean(x)
y_mean = np.nanmean(y)

#np.nanmean((x - x_mean) * (y - y_mean)) 
covariance = np.nanmean((x-x_mean)*(y-y_mean))
    
x_var = 1.0 / (len(x)-1) * np.nansum((x-x_mean)**2) # Make it consistent with Matlab's nanvar (division by len(x)-1, not len(x)))
y_var = 1.0 / (len(y)-1) * np.nansum((y-y_mean)**2)
    
CCC = (2*covariance) / (x_var + y_var + (x_mean-y_mean)**2)

x_std = np.sqrt(x_var)
y_std = np.sqrt(y_var)
PCC = covariance / (x_std * y_std)
    
RMSE = np.sqrt(np.nanmean((x - y)**2))
    
CCC_max.append(CCC)
PCC_max.append(PCC)
        
  

# CCC & PCC for valence 

In [ ]:
  

    
x = pred_V.reshape(len(Devel_Y),)


y = Devel_Y[:,1]

x_mean = np.nanmean(x)
y_mean = np.nanmean(y)

#np.nanmean((x - x_mean) * (y - y_mean)) 
covariance = np.nanmean((x-x_mean)*(y-y_mean))
    
x_var = 1.0 / (len(x)-1) * np.nansum((x-x_mean)**2) # Make it consistent with Matlab's nanvar (division by len(x)-1, not len(x)))
y_var = 1.0 / (len(y)-1) * np.nansum((y-y_mean)**2)
    
CCC = (2*covariance) / (x_var + y_var + (x_mean-y_mean)**2)

x_std = np.sqrt(x_var)
y_std = np.sqrt(y_var)
PCC = covariance / (x_std * y_std)
    
RMSE = np.sqrt(np.nanmean((x - y)**2))
    
CCC_max1.append(CCC)
PCC_max1.append(PCC)
        
    

 

# CCC & PCC for liking

In [ ]:
   

    
x = pred_L.reshape(len(Devel_Y),)


y = Devel_Y[:,2]

x_mean = np.nanmean(x)
y_mean = np.nanmean(y)

#np.nanmean((x - x_mean) * (y - y_mean)) 
covariance = np.nanmean((x-x_mean)*(y-y_mean))
    
x_var = 1.0 / (len(x)-1) * np.nansum((x-x_mean)**2) # Make it consistent with Matlab's nanvar (division by len(x)-1, not len(x)))
y_var = 1.0 / (len(y)-1) * np.nansum((y-y_mean)**2)
    
CCC = (2*covariance) / (x_var + y_var + (x_mean-y_mean)**2)

x_std = np.sqrt(x_var)
y_std = np.sqrt(y_var)
PCC = covariance / (x_std * y_std)
    
RMSE = np.sqrt(np.nanmean((x - y)**2))
    
CCC_max2.append(CCC)
PCC_max2.append(PCC)

# Writing the files

In [ ]:
CCC_A = max(CCC_max)
PCC_A = max(PCC_max)

CCC_V = max(CCC_max1)
PCC_V = max(PCC_max1)    
        
CCC_L = max(CCC_max2)
PCC_L = max(PCC_max2) 

In [1]:
with open("multi_results_ccc_timestep_8_Arosal.txt", 'a') as myfile:
    myfile.write(str(CCC_A) + '\n')
with open("multi_results_pcc_timestep_8_Arosal.txt", 'a') as myfile:
    myfile.write(str(PCC_A) + '\n')

        
with open("multi_results_ccc_timestep_8_valence.txt", 'a') as myfile:
    myfile.write(str(CCC_V) + '\n')
with open("multi_results_pcc_timestep_8_valence.txt", 'a') as myfile:
    myfile.write(str(PCC_V) + '\n')   
        

with open("multi_results_ccc_timestep_8_liking.txt", 'a') as myfile:
    myfile.write(str(CCC_L) + '\n')
with open("multi_results_pcc_timestep_8_liking.txt", 'a') as myfile:
    myfile.write(str(PCC_L) + '\n')        
        
        
    

    
    
    
    

Inside run_baseline.py
3
argv
['c:\\users\\abhijit\\anaconda3\\envs\\tensorflow-gpu\\lib\\site-packages\\ipykernel_launcher.py', '-f', 'C:\\Users\\abhijit\\AppData\\Roaming\\jupyter\\runtime\\kernel-4213064c-94d2-4fb9-8e37-2b2f2f371a9f.json']
delay
2.2
i am going
['Train_01.csv', 'Train_02.csv', 'Train_03.csv', 'Train_04.csv', 'Train_05.csv', 'Train_06.csv', 'Train_07.csv', 'Train_08.csv', 'Train_09.csv', 'Train_10.csv', 'Train_11.csv', 'Train_12.csv', 'Train_13.csv', 'Train_14.csv', 'Train_15.csv', 'Train_16.csv', 'Train_17.csv', 'Train_18.csv', 'Train_19.csv', 'Train_20.csv', 'Train_21.csv', 'Train_22.csv', 'Train_23.csv', 'Train_24.csv', 'Train_25.csv', 'Train_26.csv', 'Train_27.csv', 'Train_28.csv', 'Train_29.csv', 'Train_30.csv', 'Train_31.csv', 'Train_32.csv', 'Train_33.csv', 'Train_34.csv']


c:\users\abhijit\anaconda3\envs\tensorflow-gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 8, 600)            12292800  
_________________________________________________________________
lstm_2 (LSTM)                (None, 8, 200)            640800    
_________________________________________________________________
lstm_3 (LSTM)                (None, 8, 200)            320800    
_________________________________________________________________
lstm_4 (LSTM)                (None, 8, 200)            320800    
_________________________________________________________________
lstm_5 (LSTM)                (None, 8, 200)            320800    
_________________________________________________________________
lstm_6 (LSTM)                (None, 8, 200)            320800    
_________________________________________________________________
lstm_7 (LSTM)                (None, 200)               320800    
__________

Train on 56085 samples, validate on 22720 samples
Epoch 1/1
56085/56085 [==============================] - 158s 3ms/step - loss: 0.0029 - acc: 0.7547 - val_loss: 0.0151 - val_acc: 0.5239
[0.47993689576693416, 0.43676435228911936, 0.47079254179990204, 0.4631249787676208, 0.4852944121600839]
[0.42738626811957325, 0.3919191969240107, 0.47087767901814004, 0.42449987093876235, 0.4386628865199702]
[0.07238362668576107, 0.08932813580538743, 0.13691383328326445, 0.07075684546046643, 0.11625398384050546]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_36 (LSTM)               (None, 8, 600)            12292800  
_________________________________________________________________
lstm_37 (LSTM)               (None, 8, 200)            640800    
_________________________________________________________________
lstm_38 (LSTM)               (None, 8, 200)            320800    
_____________________________________

Train on 56085 samples, validate on 22720 samples
Epoch 1/1
56085/56085 [==============================] - 157s 3ms/step - loss: 0.0028 - acc: 0.7575 - val_loss: 0.0155 - val_acc: 0.5275
[0.47993689576693416, 0.43676435228911936, 0.47079254179990204, 0.4631249787676208, 0.4852944121600839, 0.4948668512819527, 0.44614098701113913, 0.46972266170896054, 0.44306124327377605]
[0.42738626811957325, 0.3919191969240107, 0.47087767901814004, 0.42449987093876235, 0.4386628865199702, 0.4629844956180904, 0.4536070993210443, 0.42808469368763824, 0.4136862827102464]
[0.07238362668576107, 0.08932813580538743, 0.13691383328326445, 0.07075684546046643, 0.11625398384050546, 0.12907707016500508, 0.03665942698665084, 0.13428541523453189, 0.07540504183015921]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_64 (LSTM)               (None, 8, 600)            12292800  
_____________________________________________________